# Hadi Babalou - 810199380

Artificial Intelligence - CA#03: *Game* - Spring 2023 \
In this notebook, we will simulate othello game. \
This game is played by two agents. One of them uses minimax algorithm with alpha-beta pruning and the other one plays randomly. 

In [5]:
import random
import time
import turtle
import copy
import math

nodes_expanded = 0

## GUI


The ```OthelloUI``` class is responsible for the GUI of the game. this class uses ```turtle``` library to draw the game board and the pieces.


In [6]:
class OthelloUI:
    def __init__(self, board_size=6, square_size=60):
        self.board_size = board_size
        self.square_size = square_size
        self.screen = turtle.Screen()
        self.screen.setup(self.board_size * self.square_size + 50, self.board_size * self.square_size + 50)
        self.screen.bgcolor('white')
        self.screen.title('Othello')
        self.pen = turtle.Turtle()
        self.pen.hideturtle()
        self.pen.speed(0)
        turtle.tracer(0, 0)

    def draw_board(self, board):
        self.pen.penup()
        x, y = -self.board_size / 2 * self.square_size, self.board_size / 2 * self.square_size
        for i in range(self.board_size):
            self.pen.penup()
            for j in range(self.board_size):
                self.pen.goto(x + j * self.square_size, y - i * self.square_size)
                self.pen.pendown()
                self.pen.fillcolor('green')
                self.pen.begin_fill()
                self.pen.setheading(0)
                for _ in range(4):
                    self.pen.forward(self.square_size)
                    self.pen.right(90)
                self.pen.penup()
                self.pen.end_fill()
                self.pen.goto(x + j * self.square_size + self.square_size / 2,
                              y - i * self.square_size - self.square_size + 5)
                if board[i][j] == 1:
                    self.pen.fillcolor('white')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()
                elif board[i][j] == -1:
                    self.pen.fillcolor('black')
                    self.pen.begin_fill()
                    self.pen.circle(self.square_size / 2 - 5)
                    self.pen.end_fill()

        turtle.update()

## Game


The ```Othello``` class is our main class. It is responsible for the game logic and the game flow. \
The ```Othello``` class has following attributes:
* ```size```: size of the board
* ```ui```: instance of ```OthelloUI``` class or ```None``` if we don't want to use GUI.
* ```board```: the board itself as a 2D array of {-1, 0, 1}. -1 means black, 0 means empty, and 1 means white.
* ```current_turn```: the current turn. -1 means black, and 1 means white.
* ```minimax_depth```: the maximum depth of the minimax algorithm.
* ```prune```: whether to use alpha-beta pruning or not.

The ```Othello``` class has following methods:
* ```restart_game```: restarts the game.
* ```get_winner```: returns the winner of the game at the current state.
* ```get_valid_moves```: gets player turn and returns a list of valid moves for that player.
* ```make_move```: gets player turn and a move - a tuple of (row, column) - and makes the move.
* ```get_cpu_move```: returns a random move from the list of valid moves for the random agent.
* ```get_human_move```: uses ```minimax``` method to get the best move for the human agent.
* ```minimax```: gets player turn, current depth, alpha, and beta and recursively calculates the best move for that player. 
* ```get_score```: This is our evaluation function. It calculates the score of the board for the human agent and uses it as the heuristic value for the minimax algorithm.
* ```terminal_test```: checks if the game is over or not.
* ```play```: starts the game. It uses ```get_human_move``` and ```get_cpu_move``` to get the moves and uses ```make_move``` to make the moves. It also uses ```ui``` to draw the board and the pieces.


In [7]:
class Othello:
    def __init__(self, ui, minimax_depth=1, prune=True):
        self.size = 6
        self.ui = OthelloUI(self.size) if ui else None
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
        self.current_turn = random.choice([1, -1])
        self.minimax_depth = minimax_depth
        self.prune = prune

    def restart_game(self):
        self.board = [[0 for _ in range(self.size)] for _ in range(self.size)]
        self.board[int(self.size / 2) - 1][int(self.size / 2) - 1] = self.board[int(self.size / 2)][
            int(self.size / 2)] = 1
        self.board[int(self.size / 2) - 1][int(self.size / 2)] = self.board[int(self.size / 2)][
            int(self.size / 2) - 1] = -1
        self.current_turn = random.choice([1, -1])

    def get_winner(self):
        white_count = sum([row.count(1) for row in self.board])
        black_count = sum([row.count(-1) for row in self.board])
        if white_count > black_count:
            return 1
        elif white_count < black_count:
            return -1
        else:
            return 0

    def get_valid_moves(self, player):
        moves = set()
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0:
                    for di in [-1, 0, 1]:
                        for dj in [-1, 0, 1]:
                            if di == 0 and dj == 0:
                                continue
                            x, y = i, j
                            captured = []
                            while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == -player:
                                captured.append((x + di, y + dj))
                                x += di
                                y += dj
                            if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][
                                    y + dj] == player and len(captured) > 0:
                                moves.add((i, j))
        return list(moves)

    def make_move(self, player, move):
        i, j = move
        self.board[i][j] = player
        for di in [-1, 0, 1]:
            for dj in [-1, 0, 1]:
                if di == 0 and dj == 0:
                    continue
                x, y = i, j
                captured = []
                while 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == -player:
                    captured.append((x + di, y + dj))
                    x += di
                    y += dj
                if 0 <= x + di < self.size and 0 <= y + dj < self.size and self.board[x + di][y + dj] == player:
                    for (cx, cy) in captured:
                        self.board[cx][cy] = player

    def get_cpu_move(self):
        moves = self.get_valid_moves(-1)
        if len(moves) == 0:
            return None
        return random.choice(moves)

    def get_human_move(self):
        move, score, _ = self.minimax(self.minimax_depth, 1, -math.inf, math.inf)
        return move
        
    def minimax(self, depth, player, alpha, beta):
        global nodes_expanded
        nodes_expanded += 1
        if depth == 0 or self.terminal_test():
            return None, self.get_score(player, self.board), depth
        
        board_copy = copy.deepcopy(self.board)
        best_move = None
        best_score_depth = -1

        if player == 1:
            max_score = -math.inf
            for move in self.get_valid_moves(player):
                self.make_move(player, move)
                _, score, rec_depth = self.minimax(depth - 1, -player, alpha, beta)

                if score > max_score:
                    max_score = score
                    best_move = move
                    best_score_depth = rec_depth
                    alpha = max(alpha, max_score)
                    if self.prune and max_score >= beta:
                        break

                elif score == max_score:
                    if rec_depth > best_score_depth:
                        best_move = move
                        best_score_depth = rec_depth

                self.board = copy.deepcopy(board_copy)

            return best_move, max_score, best_score_depth
        

        elif player == -1:
            min_score = math.inf
            for move in self.get_valid_moves(player):
                self.make_move(player, move)
                _, score, rec_depth = self.minimax(depth - 1, -player, alpha, beta)
                
                if score < min_score:
                    min_score = score
                    best_move = move
                    best_score_depth = rec_depth
                    beta = min(beta, min_score)
                    if self.prune and min_score <= alpha:
                        break

                elif score == min_score:
                    if rec_depth > best_score_depth:
                        best_move = move
                        best_score_depth = rec_depth

                self.board = copy.deepcopy(board_copy)

            return best_move, min_score, best_score_depth    
    
    def get_score(self, player, board):
        score = 0
        for i in range(self.size):
            for j in range(self.size):
                score += board[i][j]
        return score

    def set_minimax_depth(self, depth):
        self.minimax_depth = depth

    def set_prune(self, prune):
        self.prune = prune

    def terminal_test(self):
        return len(self.get_valid_moves(1)) == 0 and len(self.get_valid_moves(-1)) == 0

    def play(self):
        winner = None
        round_num = 1
        global nodes_expanded
        nodes_expanded = 0
        while not self.terminal_test():
            if round_num > 40:
                self.restart_game()
                return 1
            if self.current_turn == 1:
                move = self.get_human_move()
                if move:
                    self.make_move(self.current_turn, move)
            else:
                move = self.get_cpu_move()
                if move:
                    self.make_move(self.current_turn, move)
            self.current_turn = -self.current_turn
            if self.ui:
                self.ui.draw_board(self.board)
                time.sleep(0.25)
            round_num += 1
        winner = self.get_winner()
        self.restart_game()
        return winner


## Heuristic Function


Our heuristic function is very simple and it's implemented in the ```get_score``` method of the ```Othello``` class. \
The score is calculated by the following formula: 
$$ \text{score} = \text{number of white pieces} - \text{number of black pieces} $$
This function is very simple and it doesn't consider the position of the pieces, but because the other agent plays randomly, this function is enough to win the game. \
To improve the performance of the algorithm, we can use a more complex heuristic function that considers the position of the pieces. We can use the following formula for example: \
    $$ \text{score} = \text{number of white pieces} - \text{number of black pieces} + w_{corner} * (\text{number of white pieces in the corners} - \text{number of black pieces in the corners}) + w_{edge} * (\text{number of white pieces in the edges} - \text{number of black pieces in the edges}) $$

## Simulation and Results


First we play 1 game with GUI and depth of 3 to see how the game works.


In [8]:
MINIMAX_DEPTH = 3
UI = True
PRUNE = True

othello = Othello(ui=UI, minimax_depth=MINIMAX_DEPTH, prune=PRUNE)
start = time.time()
winner = othello.play()
print(f'Winner of game: {winner}, nodes expanded: {nodes_expanded}, time: {time.time() - start:.3f} seconds', end="")
end = time.time()

Winner of game: 1, nodes expanded: 1660, time: 9.279 seconds

Now we play multiple games with different depths without GUI and prunning to see how the performance changes.

In [9]:
MINIMAX_DEPTH = 1
UI = False
PRUNE = False
REPEAT = 150

othello = Othello(ui=UI, minimax_depth=MINIMAX_DEPTH, prune=PRUNE)
winners = [0, 0, 0]
start = time.time()
nodes_expanded_sum = 0
for i in range(REPEAT):
    winner = othello.play()
    print('\r', end="")
    print(100 * ' ', end="")
    print('\r', end="")
    print(f'Winner of game {i+1}: {winner}, nodes expanded: {nodes_expanded}, time: {time.time() - start:.3f} seconds', end="")
    winners[winner + 1] += 1
    nodes_expanded_sum += nodes_expanded
end = time.time()

print('\r', end="")
print(100 * ' ', end="")
print('\r', end="")
print(f'MINIMAX_DEPTH: {MINIMAX_DEPTH}')
print(f'Time taken: {end - start:.3f} seconds')
print(f'Average time taken per game: {(end - start) / REPEAT:.3f} seconds')
print(f'White wins: {winners[2]}')
print(f'Black wins: {winners[0]}')
print(f'Draws: {winners[1]}')
print(f'White win percentage: {winners[2] / (winners[0] + winners[2]) * 100:.2f}%')
print(f'Nodes expanded mean: {nodes_expanded_sum / REPEAT:.2f}')

MINIMAX_DEPTH: 1                                                                                    
Time taken: 2.180 seconds
Average time taken per game: 0.015 seconds
White wins: 86
Black wins: 57
Draws: 7
White win percentage: 60.14%
Nodes expanded mean: 94.43


In [10]:
MINIMAX_DEPTH = 3
UI = False
PRUNE = False
REPEAT = 150

othello = Othello(ui=UI, minimax_depth=MINIMAX_DEPTH, prune=PRUNE)
winners = [0, 0, 0]
start = time.time()
nodes_expanded_sum = 0
for i in range(REPEAT):
    winner = othello.play()
    print('\r', end="")
    print(100 * ' ', end="")
    print('\r', end="")
    print(f'Winner of game {i+1}: {winner}, nodes expanded: {nodes_expanded}, time: {time.time() - start:.3f} seconds', end="")
    winners[winner + 1] += 1
    nodes_expanded_sum += nodes_expanded
end = time.time()

print('\r', end="")
print(100 * ' ', end="")
print('\r', end="")
print(f'MINIMAX_DEPTH: {MINIMAX_DEPTH}')
print(f'Time taken: {end - start:.3f} seconds')
print(f'Average time taken per game: {(end - start) / REPEAT:.3f} seconds')
print(f'White wins: {winners[2]}')
print(f'Black wins: {winners[0]}')
print(f'Draws: {winners[1]}')
print(f'White win percentage: {winners[2] / (winners[0] + winners[2]) * 100:.2f}%')
print(f'Nodes expanded mean: {nodes_expanded_sum / REPEAT:.2f}')

MINIMAX_DEPTH: 3                                                                                    
Time taken: 33.711 seconds
Average time taken per game: 0.225 seconds
White wins: 119
Black wins: 25
Draws: 6
White win percentage: 82.64%
Nodes expanded mean: 3122.71


In [11]:
MINIMAX_DEPTH = 5
UI = False
PRUNE = False
REPEAT = 100

othello = Othello(ui=UI, minimax_depth=MINIMAX_DEPTH, prune=PRUNE)
winners = [0, 0, 0]
start = time.time()
nodes_expanded_sum = 0
for i in range(REPEAT):
    winner = othello.play()
    print('\r', end="")
    print(100 * ' ', end="")
    print('\r', end="")
    print(f'Winner of game {i+1}: {winner}, nodes expanded: {nodes_expanded}, time: {time.time() - start:.3f} seconds', end="")
    winners[winner + 1] += 1
    nodes_expanded_sum += nodes_expanded
end = time.time()

print('\r', end="")
print(100 * ' ', end="")
print('\r', end="")
print(f'MINIMAX_DEPTH: {MINIMAX_DEPTH}')
print(f'Time taken: {end - start:.3f} seconds')
print(f'Average time taken per game: {(end - start) / REPEAT:.3f} seconds')
print(f'White wins: {winners[2]}')
print(f'Black wins: {winners[0]}')
print(f'Draws: {winners[1]}')
print(f'White win percentage: {winners[2] / (winners[0] + winners[2]) * 100:.2f}%')
print(f'Nodes expanded mean: {nodes_expanded_sum / REPEAT:.2f}')

MINIMAX_DEPTH: 5                                                                                    
Time taken: 795.892 seconds
Average time taken per game: 7.959 seconds
White wins: 96
Black wins: 3
Draws: 1
White win percentage: 96.97%
Nodes expanded mean: 111867.57


Now we enable prunning to see how it affects the performance.

In [12]:
MINIMAX_DEPTH = 1
UI = False
PRUNE = True
REPEAT = 150

othello = Othello(ui=UI, minimax_depth=MINIMAX_DEPTH, prune=PRUNE)
winners = [0, 0, 0]
start = time.time()
nodes_expanded_sum = 0
for i in range(REPEAT):
    winner = othello.play()
    print('\r', end="")
    print(100 * ' ', end="")
    print('\r', end="")
    print(f'Winner of game {i+1}: {winner}, nodes expanded: {nodes_expanded}, time: {time.time() - start:.3f} seconds', end="")
    winners[winner + 1] += 1
    nodes_expanded_sum += nodes_expanded
end = time.time()

print('\r', end="")
print(100 * ' ', end="")
print('\r', end="")
print(f'MINIMAX_DEPTH: {MINIMAX_DEPTH}')
print(f'Time taken: {end - start:.3f} seconds')
print(f'Average time taken per game: {(end - start) / REPEAT:.3f} seconds')
print(f'White wins: {winners[2]}')
print(f'Black wins: {winners[0]}')
print(f'Draws: {winners[1]}')
print(f'White win percentage: {winners[2] / (winners[0] + winners[2]) * 100:.2f}%')
print(f'Nodes expanded mean: {nodes_expanded_sum / REPEAT:.2f}')

MINIMAX_DEPTH: 1                                                                                    
Time taken: 1.584 seconds
Average time taken per game: 0.011 seconds
White wins: 83
Black wins: 57
Draws: 10
White win percentage: 59.29%
Nodes expanded mean: 94.87


In [13]:
MINIMAX_DEPTH = 3
UI = False
PRUNE = True
REPEAT = 150

othello = Othello(ui=UI, minimax_depth=MINIMAX_DEPTH, prune=PRUNE)
winners = [0, 0, 0]
start = time.time()
nodes_expanded_sum = 0
for i in range(REPEAT):
    winner = othello.play()
    print('\r', end="")
    print(100 * ' ', end="")
    print('\r', end="")
    print(f'Winner of game {i+1}: {winner}, nodes expanded: {nodes_expanded}, time: {time.time() - start:.3f} seconds', end="")
    winners[winner + 1] += 1
    nodes_expanded_sum += nodes_expanded
end = time.time()

print('\r', end="")
print(100 * ' ', end="")
print('\r', end="")
print(f'MINIMAX_DEPTH: {MINIMAX_DEPTH}')
print(f'Time taken: {end - start:.3f} seconds')
print(f'Average time taken per game: {(end - start) / REPEAT:.3f} seconds')
print(f'White wins: {winners[2]}')
print(f'Black wins: {winners[0]}')
print(f'Draws: {winners[1]}')
print(f'White win percentage: {winners[2] / (winners[0] + winners[2]) * 100:.2f}%')
print(f'Nodes expanded mean: {nodes_expanded_sum / REPEAT:.2f}')

MINIMAX_DEPTH: 3                                                                                    
Time taken: 14.748 seconds
Average time taken per game: 0.098 seconds
White wins: 107
Black wins: 37
Draws: 6
White win percentage: 74.31%
Nodes expanded mean: 1245.87


In [14]:
MINIMAX_DEPTH = 5
UI = False
PRUNE = True
REPEAT = 100

othello = Othello(ui=UI, minimax_depth=MINIMAX_DEPTH, prune=PRUNE)
winners = [0, 0, 0]
start = time.time()
nodes_expanded_sum = 0
for i in range(REPEAT):
    winner = othello.play()
    print('\r', end="")
    print(100 * ' ', end="")
    print('\r', end="")
    print(f'Winner of game {i+1}: {winner}, nodes expanded: {nodes_expanded}, time: {time.time() - start:.3f} seconds', end="")
    winners[winner + 1] += 1
    nodes_expanded_sum += nodes_expanded
end = time.time()

print('\r', end="")
print(100 * ' ', end="")
print('\r', end="")
print(f'MINIMAX_DEPTH: {MINIMAX_DEPTH}')
print(f'Time taken: {end - start:.3f} seconds')
print(f'Average time taken per game: {(end - start) / REPEAT:.3f} seconds')
print(f'White wins: {winners[2]}')
print(f'Black wins: {winners[0]}')
print(f'Draws: {winners[1]}')
print(f'White win percentage: {winners[2] / (winners[0] + winners[2]) * 100:.2f}%')
print(f'Nodes expanded mean: {nodes_expanded_sum / REPEAT:.2f}')

MINIMAX_DEPTH: 5                                                                                    
Time taken: 120.601 seconds
Average time taken per game: 1.206 seconds
White wins: 94
Black wins: 4
Draws: 2
White win percentage: 95.92%
Nodes expanded mean: 13604.04


In [15]:
MINIMAX_DEPTH = 7
UI = False
PRUNE = True
REPEAT = 100

othello = Othello(ui=UI, minimax_depth=MINIMAX_DEPTH, prune=PRUNE)
winners = [0, 0, 0]
start = time.time()
nodes_expanded_sum = 0
for i in range(REPEAT):
    winner = othello.play()
    print('\r', end="")
    print(100 * ' ', end="")
    print('\r', end="")
    print(f'Winner of game {i+1}: {winner}, nodes expanded: {nodes_expanded}, time: {time.time() - start:.3f} seconds', end="")
    winners[winner + 1] += 1
    nodes_expanded_sum += nodes_expanded
end = time.time()

print('\r', end="")
print(100 * ' ', end="")
print('\r', end="")
print(f'MINIMAX_DEPTH: {MINIMAX_DEPTH}')
print(f'Time taken: {end - start:.3f} seconds')
print(f'Average time taken per game: {(end - start) / REPEAT:.3f} seconds')
print(f'White wins: {winners[2]}')
print(f'Black wins: {winners[0]}')
print(f'Draws: {winners[1]}')
print(f'White win percentage: {winners[2] / (winners[0] + winners[2]) * 100:.2f}%')
print(f'Nodes expanded mean: {nodes_expanded_sum / REPEAT:.2f}')

MINIMAX_DEPTH: 7                                                                                    
Time taken: 1285.036 seconds
Average time taken per game: 12.850 seconds
White wins: 95
Black wins: 5
Draws: 0
White win percentage: 95.00%
Nodes expanded mean: 136185.31


## Questions


### 1. How did you design your heuristic function and why?
A good heuristic function should be fast to calculate and it should be able to estimate the score of the board for a player. \
details of the heuristic function is explained in the heuristic function section. 


### 2. What are the effects of the depth of the search tree on the performance of the algorithm?
As we can see in the results, by increasing the depth of the search tree, the number of nodes that the algorithm has to visit increases and therefore the time that the algorithm needs to find the best move increases. \
We can see that the time and number of nodes increases exponentially with the depth of the search tree. \
But in exchange, we can win the game with a higher probability because the algorithm can search deeper in the search tree and find better moves.


### 3. Can we change the order of visiting the children of a node in the minimax algorithm so that we can prune more branches? If so, how?
In alpha-beta pruning, the order of visiting child nodes is crucial for maximizing the number of nodes pruned. The basic idea is to visit the most promising nodes first, i.e., those that are most likely to lead to a successful outcome. \
One common heuristic is to order the child nodes based on their evaluation function values. In particular, the best child node is visited first, followed by the second-best, and so on. This is known as the "static ordering" strategy, as the order of visiting the child nodes is fixed before the search begins. \
Another approach is to use a "dynamic ordering" strategy, which updates the order of child nodes during the search based on the current state of the game. For example, if a child node has been found to be promising, it can be moved to the front of the order to be explored first. \
There are also more sophisticated methods that use machine learning or other techniques to learn the best ordering of child nodes. These methods can be highly effective, but they may require more computational resources and training data. \
Ultimately, the choice of ordering strategy will depend on the specific problem domain and the available computational resources. However, by carefully considering the order of visiting child nodes, it is possible to prune a maximum number of nodes and speed up the search process in alpha-beta pruning.


### 4. Explain what is "Branching Factor" and how it changes during the process of the game?
The branching factor is a measure of the number of possible moves that can be made at each level of a search tree. In the context of game playing algorithms like Minimax with Alpha-Beta Pruning, the branching factor refers to the number of legal moves available to the player at any given point in the game. \
When playing Othello using Minimax with Alpha-Beta Pruning, the branching factor will change over time as the game progresses. In the beginning, the branching factor will be relatively high, as there are many possible moves available to both players. However, as the game progresses and more pieces are placed on the board, the number of legal moves available will decrease, and the branching factor will decrease accordingly. \
In the early stages of the game, the branching factor can be quite high, often ranging from 20 to 30 moves. However, as the game progresses and the board becomes more crowded, the branching factor will decrease, sometimes to as few as 5 to 10 moves per turn. \
Minimax with Alpha-Beta Pruning is designed to take advantage of this changing branching factor by exploring the most promising moves first and pruning branches of the tree that are unlikely to lead to a successful outcome. By carefully choosing which branches to explore, this algorithm can effectively search the game tree and find a strong move while minimizing the number of nodes it needs to explore. This is particularly useful in games like Othello, where the branching factor can be quite large in the early game but decreases as the game progresses.

### 5. Why pruning makes the algorithm faster without decreasing the quality of the solution?
Pruning is a technique used in the minimax algorithm with alpha-beta pruning to reduce the number of nodes that need to be evaluated during the search process. It works by eliminating parts of the search tree that are known to be irrelevant to the final solution, thereby reducing the computational time required to find the optimal solution. \
The minimax algorithm with alpha-beta pruning is a depth-first search algorithm that evaluates every possible move from the current state of the game. During the search process, the algorithm maintains two values: alpha, which represents the best value found so far for the maximizing player, and beta, which represents the best value found so far for the minimizing player. These values are used to prune branches of the search tree that cannot lead to a better solution than what has already been found. \
The benefit of pruning is that it eliminates large portions of the search space that are guaranteed to be suboptimal. By reducing the number of nodes that need to be evaluated, pruning significantly reduces the computational time required to find the optimal solution. \
Importantly, pruning does not decrease the quality of the solution because it only eliminates nodes that are guaranteed to be suboptimal. The algorithm will still explore all of the nodes that are relevant to the final solution, ensuring that the optimal solution is found. \
Furthermore, the effectiveness of alpha-beta pruning increases as the depth of the search tree increases. This is because the number of nodes that need to be evaluated grows exponentially with the depth of the tree, making it increasingly important to eliminate irrelevant nodes.


### 6. Why when the other agent plays randomly, using the minimax isn't the optimal solution? which algorithm is better in this case?
When the opposing agent plays randomly, using the minimax algorithm with alpha-beta pruning may not be the optimal solution. This is because the minimax algorithm assumes that the opponent is playing optimally, and therefore evaluates every possible move as if the opponent is trying to make the best move possible. However, if the opponent is playing randomly, the minimax algorithm may waste a lot of computational resources evaluating moves that are not optimal in practice. \
In this case, a more appropriate algorithm to use might be a Monte Carlo Tree Search (MCTS) algorithm. MCTS is a heuristic-based search algorithm that is particularly effective when the opponent is playing randomly or sub-optimally. \
MCTS works by building a tree of possible moves and then simulating random games from the current node to the end of the game. The algorithm uses the results of these simulations to guide the search towards the most promising moves. Unlike minimax, MCTS does not assume that the opponent is playing optimally and instead focuses on exploring the most promising branches of the search tree. \
MCTS can be a particularly effective algorithm when playing against opponents who are playing randomly or sub-optimally because it does not waste computational resources evaluating moves that are not likely to be played by the opponent. Instead, it focuses on exploring the most promising branches of the search tree based on the results of simulated games. \
In summary, when the opposing agent is playing randomly, using the minimax algorithm with alpha-beta pruning may not be the optimal solution. Instead, a Monte Carlo Tree Search algorithm may be more appropriate due to its ability to efficiently explore the most promising branches of the search tree based on the results of simulated games.
